__Ideas for how to make this work__

- First assignment should be a dummy, but still graded use

- Suggest that students open canvas in 2 tabs/windows so can see original

- Video walk through of an assignment

- Have a checkbox before turn in any feedback that says "I have read my response pretending that I am the receipient and believe that the feedback is accurate, fair, and presented with [kindness? empathy?]"

- Tell that this is work in progress. Precise proportions of each grade may vary (e.g., amount for completion vs assigned). Always will be with an eye to students who complete everything pass

- My feedback / examples etc should just be loaded into the feedback they see when they turn in the first assigment

- Look through mdp materials for articles to read on constructive feedback



- Student assigned grades should be in terms of A, B, C, D, F. I will determine how to translate into points



https://canvas.instructure.com/doc/api/peer_reviews.html

In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')


import pandas as pd
pd.options.display.max_rows = 999

import datetime

from CanvasHacks import environment
# from CanvasHacks.RequestTools import *
# from CanvasHacks.UrlTools import *
from CanvasHacks.Configuration import InteractiveConfiguration
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT

# File system
# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator, create_folder
from CanvasHacks.TimeTools import getDateForMakingFileName
from CanvasHacks.QuizReportFileTools import load_new


# Canvas api
from canvasapi import Canvas
from canvasapi.quiz import QuizReport, Quiz
from canvasapi.requester import Requester
from canvasapi.conversation import Conversation

# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)
requester = Requester(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)

# Configuration
from CanvasHacks.PeerReviewed.Definitions import Review, InitialWork, MetaReview, Unit #Assignment

# Exceptions
from CanvasHacks.Errors.review_pairings import AlreadyAssigned, SubmissionIncomplete

# Models
from CanvasHacks.Models.student import Student
from CanvasHacks.Models.student import student_from_canvas_user, ensure_student

# Notifications
# from CanvasHacks.PeerReviewed.Notifications import notify_student
# from CanvasHacks.Messaging.Messengers import make_prompt_and_response, make_notice, metareview_send_message_to_reviewers, review_send_message_to_reviewers
from CanvasHacks.Models.student import get_first_name

# Plotting
from CanvasHacks.VisualizationTools import rotate_x_labels

# Repos
from CanvasHacks.Repositories.DataManagement import DataStore
from CanvasHacks.Repositories.quizzes import QuizRepository, ReviewRepository
from CanvasHacks.Repositories.codes import AccessCodeRepo
from CanvasHacks.Logging.review_pairings import make_review_audit_file
from CanvasHacks.Repositories.reviewer_associations import assign_reviewers, AssociationRepository
from CanvasHacks.Repositories.students import StudentRepository
from CanvasHacks.Repositories.submissions import SubmissionRepository, QuizSubmissionRepository

# Storage
from CanvasHacks.DAOs.sqlite_dao import SqliteDAO

# Widgets
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments
from CanvasHacks.UploadGradeTools import make_upload_button
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser

# Main things
from CanvasHacks.SkaaSteps.SendInitialWorkToReviewer import SendInitialWorkToReviewer
from CanvasHacks.SkaaSteps.SendReviewToReviewee import SendReviewToReviewee
from CanvasHacks.SkaaSteps.SendMetareviewToReviewer import SendMetareviewToReviewer

from CanvasHacks.Models.status_record import StatusRecord

import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))

    
SEMESTER_NAME = 'S20'
LOC = '{}/Box Sync/TEACHING/Phil 305 Business ethics/Phil305 S20'.format(environment.ROOT)# placeholder for where the access codes are stored
ACCESS_CODES_FP = "{}/{}-assignment-access-codes.xlsx".format(LOC, SEMESTER_NAME)



In [ ]:
import CanvasHacks.testglobals
# Set on the global variable (which is only used in certain tests)
# CanvasHacks.testglobals.TEST
CanvasHacks.testglobals.TEST_WITH_FILE_DB = True

# Select unit

In [ ]:
make_test_selector()
make_unit_chooser()

In [ ]:
SEND = True
step1 = SendInitialWorkToReviewer(course=environment.CONFIG.course, unit=environment.CONFIG.unit, send=SEND)
step1.run(rest_timeout=5)

In [ ]:
environment.CONFIG.unit.metareview.__dict__

# Distribute peer reviews to authors and request to do metareview

In [ ]:
step1.associationRepo.data

In [ ]:
e =[]
for r in step1.associationRepo.data:
#     print(step1.studentRepo.get_student(r.assessor_id).__dict__)
    e.append(step1.studentRepo.get_student(r.assessor_id))
len(e)

In [ ]:
d = step1.work_repo.data[step1.work_repo.data.workflow_state != 'unsubmitted']
d = step1.work_repo.data[~pd.isnull(step1.work_repo.data.body)]
len(d)

In [ ]:
step1.studentRepo.get_student(170579)

In [ ]:
e

In [ ]:
SEND = True
step2 = SendReviewToReviewee(environment.CONFIG.course, environment.CONFIG.unit, send=SEND)
step2.run(rest_timeout=5)

In [ ]:
from CanvasHacks.Models.status_record import StatusRecord

review_invites = step2.dao.session.query( StatusRecord ).filter( StatusRecord.activity_id == environment.CONFIG.unit.review.id).all()
for m in review_invites:
    print(m.__dict__)
review_invites

metareview_invites = step2.dao.session.query( StatusRecord ).filter( StatusRecord.activity_id == environment.CONFIG.unit.metareview.id).all()
for m in metareview_invites:
    print(m.__dict__)

init = step2.dao.session.query( StatusRecord ).filter( StatusRecord.activity_id == environment.CONFIG.unit.initial_work.id).all()
init

# Return feedback from metareview to the person who did the peer review

In [ ]:
SEND = False
step3 = SendMetareviewToReviewer(environment.CONFIG.course, environment.CONFIG.unit, send=SEND)

In [ ]:
step3.run(rest_timeout=5, download=False)

In [ ]:
step3.notificationStatusRepo.previously_sent_results


In [ ]:
step3.studentRepo.data

In [ ]:
step3.dao.session.query(StatusRecord).all()

In [ ]:
step3.notificationStatusRepo.previously_sent_results

In [ ]:
step3.work_repo.data

In [ ]:
metareview_snt = step3.dao.session.query( StatusRecord ).filter( StatusRecord.activity_id == environment.CONFIG.unit.metareview.id).all()
metareview_snt

for m in metareview_snt:
    print(m.__dict__)

In [ ]:
# Run after the unit has been selected
canvas = environment.CONFIG.canvas
course = environment.CONFIG.course
unit = environment.CONFIG.unit

codeRepo = AccessCodeRepo(ACCESS_CODES_FP, environment.CONFIG.unit)

studentRepo = StudentRepository(environment.CONFIG.course)
studentRepo.download()

# Get data for content assignment

In [ ]:
from CanvasHacks.QuizReportFileTools import  make_quiz_repo

# from CanvasHacks.QuizReportFileTools import retrieve_quiz_data, save_downloaded_report, make_quiz_repo

# def make_quiz_repo( course, activity, save=True):
#     """Gets all student work data for the activity that's part of the assignment
#     loads it into a QuizRepository or ReviewRepository and 
#     returns the repository
#     """
#     # Get quiz submission objects
#     if isinstance(activity, Review):
#         repo = ReviewRepository(activity, course)
#     else:
#         repo = QuizRepository(activity, course)
    
#     # Download student work
#     # This will work if the 'Create Report' button has been manually clicked
#     student_work_frame = retrieve_quiz_data(repo.quiz)
    
#     if save:
#         # Want to have all the reports be formatted the same
#         # regardless of whether we manually or programmatically 
#         # downloaded them. Thus we save before doing anything to them.
#         save_downloaded_report(activity, student_work_frame)

#     # Download submissions
#     subRepo = QuizSubmissionRepository(repo.quiz)

#     # Doing the combination with submissions after saving to avoid 
#     # mismatches of new and old data
#     repo._process(student_work_frame, subRepo.frame)
    
#     return repo

## Load from file like I'm an animal

In [ ]:
# Manually from file like I'm an animal

# create_folder(unit.initial_work.folder_path)

from CanvasHacks.Repositories.quizzes import load_student_work

# Load all student data for selected assignments from csv files
assignments = []
# for folder_path in ASSIGNMENT_FOLDERS:
print("Quiz: ", )
fiter = makeDataFileIterator(unit.initial_work.folder_path)
#     frame = load_student_work(next(fiter), submissions)
try:
    while True:
        f = next(fiter)
        print("loading: ", f)
        frame = pd.read_csv(f)
#         frame = load_student_work(f, submissions)
        # add submission info and use to remove non-final attempts
#         remove_non_final_attempts(frame)
        assignments.append(frame)
except StopIteration:
    pass

student_work_frame = assignments[0]

## If loading previously processed

In [ ]:
# Run this is loading processed data from file
# Manually add student work
initial_work_repo = QuizRepository(unit.initial_work)
initial_work_repo.data = student_work_frame
initial_work_repo.set_question_columns(initial_work_repo.data)

## If loading for first time

In [ ]:
# Run this if doing manually and getting data for first time
course = canvas.get_course(unit.initial_work.course_id)

quiz = course.get_quiz(unit.initial_work.quiz_id)

# Download submissions
submissions = quiz.get_submissions()    
# the canvasapi has returned a bunch of objects
# so we convert to dicts to make easier to create dataframe
submissions = [l.__dict__ for l in list(submissions)]

# Manually add student work
initial_work_repo = QuizRepository(unit.initial_work)
initial_work_repo._process(student_work_frame, submissions)
len(initial_work_repo.data)

## The right way: programmatic download

In [ ]:
# The way its supposed to be
# Load data

initial_work_repo = make_quiz_repo(environment.CONFIG.course, unit.initial_work)

initial_work_repo.data[:3]

# Assign peer reviewers
__Carried out after the initial work due date__

## Working on CAN-35

In [ ]:
No one assigned to assess them:
    Artur Alaverdyan
    Javier Parra
    Joseph Pouldar
    Kathryn Timmer
    Saida Alexandra Khurram
    
Someone assigned to assess them them
    Mohammad Karbalaei Ali Akbar --- Berenice
    
    
No one assigned for them to assess:
    Artur Alaverdyan
    Javier Parra
    Joseph Pouldar
    Kathryn Timmer
    Saida Alexandra Khurram
    
Someone assigned for them to assess:
    Mohammad Karbalaei Ali Akbar ---- Sevada Keshishi
    

In [ ]:
def make_column_map(question_columns):
    return { question_columns[i] : 'q{}'.format(i+1) for i in range(0, len(question_columns)) }

assert(make_column_map(['long1', 'long2']) == {'long1' : 'q1', 'long2': 'q2'})

fp = "{}/Desktop/TEMPORARY/Unit 1_CA.xlsx".format(environment.ROOT)
dt = pd.read_excel(fp)

cmap = {'3031694: Explain the structure of a tragedy of the commons . Make sure you explain why it remains rational to cheat even when one knows that enough people cheating will result in catastrophe. Then explain 2 measures by which the villagers in the overfishi' : 'q1'}
dt.rename(cmap, axis=1, inplace=True)
dt.rename({'id' : 'student_id'}, axis=1, inplace=True)
dt = dt[['name', 'student_id', 'sis_id', 'submitted', 'attempt', 'q1']]
dt


quiz = course.get_quiz(unit.initial_work.quiz_id)

# Download submissions
submissions = quiz.get_submissions()    
# the canvasapi has returned a bunch of objects
# so we convert to dicts to make easier to create dataframe
submissions = [l.__dict__ for l in list(submissions)]

submissions = pd.DataFrame(submissions)
submissions.rename({'user_id' : 'student_id'}, inplace=True, axis=1)
sub_cols = ['id', 'quiz_id', 'student_id', 'submission_id', 
       'end_at', 'end_at_date', 'finished_at', 'finished_at_date', 'attempt',
       'workflow_state','has_seen_results','attempts_left', 'course_id']

submissions = submissions[sub_cols]

len(submissions)

stups = submissions[['student_id', 'attempt', 'submission_id']]
stups = {sid : (a, subid) for idx,sid, a, subid in stups.itertuples()}

def add_sub_id(student_id, attempt, stups):
    s = stups.get(student_id)
    if s[0] == attempt:
        return s[1]
    return None

dt['submission_id'] = dt.apply(lambda x: add_sub_id(x.student_id, x.attempt, stups), axis=1)
dt

filter(stups, lambda x: x[0] == 57180)

def add_sub_id(student_id, attempt, stups):
    s = stups.get(student_id)
    if s[0] == attempt:
        return s[1]
    return None

dt['submission_id'] = dt.apply(lambda x: add_sub_id(x.student_id, x.attempt, stups), axis=1)


dt.dropna( subset=[ 'submission_id' ], inplace=True )
len(dt)

def local_dt_string_to_utc( local_string ):
    """'2020-02-07T07:59:59Z'
    returns Timestamp('2020-02-06 23:59:59-0800', tz='US/Pacific')
    """
    return pd.Timestamp( local_string, tz='US/Pacific' ).tz_convert( 'utc' )


submissions.finished_at = pd.to_datetime(submissions.finished_at)

last_ran = pd.Timestamp('2020-02-05T10:10', tz='US/Pacific')

pending_subs = submissions[submissions.finished_at > last_ran]

late = dt[dt.student_id.isin(pending_subs.student_id) ]

late = late[late.student_id != 155461]
late

LATE_REVIEW_ID = 11111

late_ass_repo =  AssociationRepository(dao, LATE_REVIEW_ID)

# iewers to each submitter and store in db
associationRepo.assign_reviewers( late.student_id.tolist())


for sid in late.student_id.tolist():
    print(sid, associationRepo.get_assessee(unit.review, sid), associationRepo.get_assessor(unit.review, sid))


for sid in pending_subs.student_id.tolist():
    print(sid, associationRepo.get_assessee(unit.review, sid))
    print(sid, associationRepo.get_assessor(unit.review, sid))



dt.student_id.isin(pending_subs.student_id)

review_assigns = associationRepo.get_associations(unit.review)
review_assigns

len(assignments[0])
old = assignments[0]
old.columns

len(assignments[1])
new = assignments[1]
new.columns

new[new.student_id not in old.student_id]

a = pd.concat(assignments, sort=True)

a[:3].T

len(a)

In [ ]:
from CanvasHacks.SkaaSteps.SendInitialWorkToReviewer import SendInitialWorkToReviewer

In [ ]:
s = SendInitialWorkToReviewer()

## Make and store reviewer assignments

In [ ]:
if environment.CONFIG.is_test:
    # testing: in memory db
    dao = SqliteDAO()
    print("Connected to testing db")
else:
    db_filepath = "{}/{}-Unit-{}-review-assigns.db".format( environment.LOG_FOLDER, SEMESTER_NAME, environment.CONFIG.unit_number)
    # real: file db
    dao = SqliteDAO(db_filepath)
    dao.initialize_db_file()
    print("Connected to REAL db")

associationRepo = AssociationRepository(dao, unit.review)

In [ ]:

def make_review_audit_file(associationRepo, unit):
    """Stores all review assignments to a csv file
    for easier auditing
    """
    # Make audit file
    review_audit = []
    for rev in associationRepo.get_associations(unit.review):
        print(rev.assessor_id, rev.assessee_id)
        assessor = studentRepo.get_student(rev.assessor_id)
        assessee = studentRepo.get_student(rev.assessee_id)
        print(assessor)

        review_audit.append({
            'activity' : unit.review.name,
            'assessor_name' : assessor.short_name,
            'assessor_sis_id': assessor.sis_user_id,
            'assessor_canvas_id': assessor.id,
            'assessee_name' : assessee.short_name,
            'assessee_sis_id': assessee.sis_user_id,
            'assessee_canvas_id': assessee.id,
            'timestamp': datetime.datetime.now().isoformat()
        })

    review_audit = pd.DataFrame(review_audit)

    fp = "{}/{}-Unit{}-peer review assignments.csv".format(environment.LOG_FOLDER, getDateForMakingFileName(), environment.CONFIG.unit )
    review_audit.to_csv(fp)
    print("Audit file saved to {}".format(fp))
    # review_audit

In [ ]:
# all students who submitted the assignment
# submitters = [169908, 169955, 169957]

# submitters = list(set(initial_work_repo.data.reset_index().student_id.tolist()))

In [ ]:
# Assign reviewers to each submitter and store in db
associationRepo.assign_reviewers( initial_work_repo.submitter_ids)

if not environment.CONFIG.is_test:
    make_review_audit_file(associationRepo, environment.CONFIG.unit )

## Send initial work and instructions to reviewers

Backup plan if can't use canvas conversations is to just use email

In [ ]:
from CanvasHacks.PeerReviewed.Notifications import StudentWorkForPeerReviewMessenger

In [ ]:
SEND = False
# SEND = True
msgr = StudentWorkForPeerReviewMessenger(unit.review, studentRepo, initial_work_repo )
msgr.notify(associationRepo.data, SEND)

In [ ]:
SEND = False
# SEND = True

review_send_message_to_reviewers(associationRepo.data, studentRepo, initial_work_repo, unit.review, SEND)

In [ ]:
associationRepo.data

# Metareview

__Carried out after the review due date__

## Download student work on review

In [ ]:
# REAL VERSION

# Get work
review_repo = make_quiz_repo(environment.CONFIG.course, unit.review)
#review_repo.course = course
review_repo._fix_forgot_answers()

In [ ]:
# FROM FILE TROUBLESHOOTING VERSION
# review_repo
# review_repo.data.to_csv('/Users/adam/Desktop/TEMPORARY/unit1peerreview.csv')
review_repo = ReviewRepository(unit.review, course)
review_repo.data = pd.read_csv('/Users/adam/Desktop/TEMPORARY/unit1peerreview.csv')
review_repo.set_question_columns(review_repo.data)
review_repo._fix_forgot_answers()

## Create graphics

In [ ]:
# For me
rows = nrows=round(len(multiple_choice_names)/2)
fig, axes = plt.subplots(ncols=2, nrows=rows, figsize=(12,20))

row=0; col=0
order = ['Forgot', 'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree']
for c in multiple_choice_names:
    title = c.split(':')[1][:65]
    g = sns.countplot(d[c], order=environment.LIKERT_PLOT_ORDER, palette='plasma', ax=axes[row, col])
    g.set_xlabel(title)
    rotate_x_labels(axes[row, col])
    if col == 1:
        row += 1
        col = 0
    else:
        col += 1

fig.tight_layout()

In [ ]:
rows = len(review_repo.multiple_choice_names)
fig, axes = plt.subplots(nrows=rows, figsize=(8,50))

row=0; col=0
for c in review_repo.multiple_choice_names:
    title = c.split(':')[1][:65]
    g = sns.countplot(d[c], order=environment.LIKERT_PLOT_ORDER, palette='plasma', ax=axes[row])
    g.set_xlabel(title)
    rotate_x_labels(axes[row])
    row +=1
fig.tight_layout()

## Notify reviewed student

In [ ]:
from CanvasHacks.PeerReviewed.Notifications import FeedbackForMetareviewMessenger

SEND = False
# SEND = True

msgr = FeedbackForMetareviewMessenger(unit.metareview, studentRepo, review_repo )
msgr.notify(associationRepo.data, SEND)

In [ ]:
SEND = False
# SEND =True

# Do this separately because the relevant activity is the previous review

# Load list of ReviewAssociation objects representing who reviews whom
# review_assigns = associationRepo.get_associations(unit.review)
print("loaded {} student reviewer assignments".format(len(review_assigns)))


metareview_send_message_to_reviewers(associationRepo.data, studentRepo, review_repo, unit.metareview, SEND)

# Send late peer review submissions out for metareview

In [ ]:
newframe = load_new(unit.review)

review_repo = ReviewRepository(unit.review, course)
review_repo.data = newframe
review_repo.set_question_columns(review_repo.data)

In [ ]:

SEND = False
# SEND = True

msgr = FeedbackForMetareviewMessaging(unit.metareview, studentRepo, review_repo )
msgr.notify(associationRepo.data, SEND)

In [ ]:
SEND = False
# SEND =True

# Load list of ReviewAssociation objects representing who reviews whom
# Do this separately because the relevant activity is the previous review
review_assigns = associationRepo.get_associations(unit.review)
# prune to just the new stuff
review_assigns = [ r for r in review_assigns if r.assessor_id in review_repo.data.index]
print("loaded {} student reviewer assignments".format(len(review_assigns)))

metareview_send_message_to_reviewers(review_assigns, studentRepo, review_repo, unit.metareview, SEND)

# Closing out

__Carried out after the meta review due date__

## Content assignment

### Assign grades 

based partially on review

### Upload grades

## Review

### Assign grades

Grade review for completion

Apply grades given in review to initial submitter

sent at 2020-02-09T22:49:59Z

### Upload grades

## Metareview

### Send metareview feedback and stats to reviewer

In [ ]:
# Format the feedback given in the metareview

# Calculate a statistical summary of how the ratings they gave vs. class

# Calculate a statistical summary of ratings received vs class

### Assign grades

In [ ]:

meta_repo = make_quiz_repo(assignment.meta_review)
# Grade metareview

# Apply grades given in metareview to reviewer

### Upload  grades

(This may not be necessary if metareview is created via a survey)

## Statistical and other summary information for everyone

We upload as much anonymized info as possible so that everyone can see scoring is working. This is to help everyone improve and anchor standards

# Cron job that handles all of this automagically!

In [ ]:
# Define based on Assignment and set running

# Abberant cases

- Assigned reviewer does not turn in review

- Reviewd student does not turn in metareview

- Odd number of students turn in

- Incomplete submissions

- Multiple discsussion submissions

In [ ]:
#Auditing

review_assigns = associationRepo.get_associations(unit.review)
# review_assigns

len(review_assigns)

log = pd.read_csv("{}/2020-02-05-Unit1-peer review assignments.csv".format(environment.LOG_FOLDER))

log.set_index('assessor_canvas_id', inplace=True)
log

for ra in review_assigns:
    try:
        row = log.loc[ra.assessor_id]
        if ra.assessee_id != row.assessee_canvas_id:
            print("Log assessor {} assessee {} | DB assessee {}".format(ra.assessor_id, ra.assessee_id, row.assessee_canvas_id, ))
    except KeyError:
        print("DB assessor id not in log: ", ra.assessor_id)




In [ ]:
# AUDIT

log = pd.read_csv("{}/2020-02-05-Unit1-peer review assignments.csv".format(environment.LOG_FOLDER))

log.set_index('assessor_canvas_id', inplace=True)
log

for ra in review_assigns:
    try:
        row = log.loc[ra.assessor_id]
        if ra.assessee_id != row.assessee_canvas_id:
            print("Log assessor {} assessee {} | DB assessee {}".format(ra.assessor_id, ra.assessee_id, row.assessee_canvas_id, ))
    except KeyError:
        print("DB assessor id not in log: ", ra.assessor_id)




In [ ]:
# Various test data stores

testfileloc = "{}/Desktop/TEMPORARY/test.csv".format(environment.ROOT)
test_data_folder = "{}/tests/testdata/".format(environment.CONFIG.proj_base)
test_quiz_results = "{}/quiz results.csv".format(test_data_folder)
test_submissions_frame = "{}/test submissions.csv".format(test_data_folder)
# this is what's stored in QuizRepo.data for the initial submisison after processing
test_combined_frame ="{}/test combined.csv".format(test_data_folder)

In [ ]:
adam_id = 6417

# Test data
test_course_id = 85210

main_quiz = 165098
review_quiz = 165820
meta_review = 165821

url = 'https://canvas.csun.edu/api/v1/courses/85210/assignments/{}/submissions'

# last semester data
prev_course_id = 62657
prev_quiz = 151633

COURSE_ID = test_course_id
# COURSE_ID = prev_course_id
# QUIZ_ID = prev_quiz

TEST_STUDENT_ID = 168439

In [ ]:
review_repo.data.set_index('student_id', inplace=True)

unit.metareview.access_code = ''

pd.read_csv('/Users/adam/Box Sync/TEACHING/Phil 305 Business ethics/Student work/77522-Unit 1 Peer review/2020-02-08-Unit 1 Peer review-student-work.csv')

from CanvasHacks.Repositories.quizzes import load_student_work
# prev = "2020-02-08-Unit 1 Peer review-student-work.csv"
latest = "{}/2020-02-10-Unit 1 Peer review-student-work.csv".format(unit.review.folder_path)
latest = pd.read_csv(latest)
latest.rename( { 'id': 'student_id' }, axis=1, inplace=True )
latest.set_index('student_id', inplace=True)
latest.submitted = pd.to_datetime(latest.submitted)
latest = latest[['submitted', 'attempt']]
latest['v'] = 'latest'
# latest = load_student_work(latest)

prev = pd.read_csv('/Users/adam/Desktop/TEMPORARY/unit1peerreview.csv')
# prev = load_student_work('/Users/adam/Desktop/TEMPORARY/unit1peerreview.csv')
prev.set_index('student_id', inplace=True)
prev.submitted = pd.to_datetime(prev.submitted)
prev = prev[['submitted', 'attempt']]
prev['v'] = 'previous'

review_assigns[0].id

fs = [prev, latest]
fs.sort(key=lambda x: x.submitted.max(), reverse=True)

fs[0]

import numpy as np
assert(prev.submitted.dtype == np.datetime64)

prev.submitted = pd.to_datetime(prev.submitted)

newsubs = latest[~latest.index.isin(prev.index)]

newsubs

# Anything which happened after the last check of the data
newsubs = latest[latest.submitted > prev.submitted.max()]
newsubs

prev.submitted.max()

# Any second or later attempts that occurred after last 
# check of data
#attempts
latest[latest.attempt > 1]

In [ ]:
TEST = True
# TEST = False
USE_PREVIOUS = False

if TEST:
    environment.CONFIG.set_test()
# environment.CONFIG.set_live()

COURSE_ID = environment.CONFIG.course_ids[0]
print("Working on course: ", COURSE_ID)


# UNIT_NUMBER = 1

# Create assignments (CAN-20)

In [ ]:
from CanvasHacks.PeerReviewed.SkaaMaker import UnitDefinitionsLoader, CreationHandlerFactory
LOC = '{}/Box Sync/TEACHING/Phil 305 Business ethics/Phil305 S20'.format(environment.ROOT)# placeholder for where the access codes are stored

DETAILS_FP = "{}/Unit details.xlsx".format(LOC)

maker = UnitDefinitionsLoader()
maker.load(DETAILS_FP)
units = maker.create_units()



In [ ]:
for u in units:
    for a in u.components:
        maker = CreationHandlerFactory.make(a, environment.CONFIG.course)
        print(maker.creation_dict)
#         maker.create()

In [ ]:
# test students
STUDENT_TEST_ACCT_FP = "{}/private/test-accounts.xlsx".format(environment.CONFIG.proj_base)
student_users_df = pd.read_excel(STUDENT_TEST_ACCT_FP)
tokens = student_users_df.token.dropna().tolist()

In [ ]:
# for testing
fp = "{}/student-work.csv".format(initial_work_repo.activity.folder_path )
initial_work_repo.data = pd.read_csv(fp)

In [ ]:
DETAILS_FP = "{}/Unit details.xlsx".format(LOC)
with pd.ExcelFile(DETAILS_FP) as xls:
    unit_details = pd.read_excel(xls, 'details')
    start_dates = pd.read_excel(xls, 'startdates')

# start_dates.start = start_dates.apply(lambda x: pd.Timestamp(x.start, tz='US/Pacific'), axis=1)
start_dates.start = start_dates.apply(lambda x: pd.Timestamp(x.start), axis=1)

# Clean up the text
unit_details.activity_title = unit_details.activity_title.str.strip()

units = []
for i, row in start_dates.iterrows():
    print(row.unit)
    u = Unit([], row.unit)
    for activity_type in u.component_types:
        activity = activity_type(unit_number=u.unit_number)
#         print(activity.title_base)
        deets = unit_details[unit_details.activity_title == activity.title_base]
#         print(deets)        
        setattr(activity, 'unit_number', row.unit)
        v = {'unit_number' : row.unit}
        d = row.start
        
#         v['unlock_at'] = d + pd.Timedelta('1 minutes')
#         setattr(activity, 'unlock_at', v['unlock_at'])

#         d += pd.Timedelta('{} days'.format(deets.due_days_from_start.values[0]))
#         v['due_at'] = d - pd.Timedelta('1 minute')
#         setattr(activity, 'due_at', v['due_at'])
        
#         d += pd.Timedelta('{} days'.format(deets.lock_after_due.values[0]))
#         v['lock_at'] = d - pd.Timedelta('1 minute')
#         setattr(activity, 'lock_at', v['lock_at'])
        
#         v['points_possible'] = deets.points.values[0]
#         setattr(activity, 'points_possible', v['points_possible'])
        
#         v['description'] = activity.description
#         setattr(activity, 'description', v['description'])
        
        u.components.append(activity)
#         u.components.append( activity_type( **v) )
    units.append(u)
# units

In [ ]:
u

In [ ]:
dateframe = []
for u in units:
    for a in u.components:
        d = a.__dict__
        d['title'] = a.make_title
        dateframe.append(d)
dateframe = pd.DataFrame(dateframe)
dateframe

In [ ]:
start_dates

In [ ]:
unit_details

In [ ]:
unit_details[unit_details.activity_title == 'Content assignment']


In [ ]:
unit_details.activity_title = unit_details.activity_title.str.strip()

In [ ]:
unit_details.activity_title

In [ ]:
# Get instructions template for the assignment

In [ ]:
# Get questions template where applicable

In [ ]:
# Create the assignment

In [ ]:
# Maybe download template quiz and then just replace

acts = []
d = unit_start_date
for activity, days in main:
    v = {'unit_number' : UNIT_NUMBER}
    v['unlock_at'] = d + pd.Timedelta('1 minutes')
    d += pd.Timedelta('{} days'.format(days))
    v['due_at'] = d - pd.Timedelta('1 minute')
    # todo adjust for grace period
    v['lock_at'] = v['due_at']
    print(v)
    acts.append( activity( **v) )
    
acts
    

# todo translate to utc
unit_start_date = pd.Timestamp('2020-02-01', tz='US/Pacific')
UNIT_NUMBER = 4

main = [ (TopicalAssignment, 2),
            (InitialWork, 7),
            (Review, 2),
            (MetaReview, 2)]
forum_num_days = [
            (DiscussionForum, 7),
            (DiscussionReview, 2)]
# sum([i[1] for i in num_days])
# assert(sum([i[1] for i in num_days]) == 14)
            

In [ ]:
# Print unit dates to df so can give students a spreadsheet

# Manage test data

In [ ]:
test_quiz_results = "{}/quiz results.csv".format(test_data_folder)
test_quiz_results = pd.read_csv(test_quiz_results)
from faker import Faker
fake = Faker()
d = test_quiz_results
d.name = d.apply(lambda x: fake.name(), axis=1)
d.id = d.apply(lambda x: fake.ean(length=8), axis=1)
d.sis_id = d.apply(lambda x: fake.ean(length=8), axis=1)
d.section_id = d.apply(lambda x: fake.ean(length=8), axis=1)
d.section_sis_id = d.apply(lambda x: fake.ean(length=8), axis=1)

# d.to_csv(test_quiz_results)

d.submission_id = d.apply(lambda x: fake.ean(length=8), axis=1)
d.validation_token = d.apply(lambda x: fake.sha1(), axis=1)
# submissions_frame.drop(['_requester', 'attributes', 'html_url', 'result_url'], inplace=True, axis=1)
d.id = d.apply(lambda x: fake.ean(length=8), axis=1)
# d.to_csv(test_quiz_results)
d.to_csv(test_combined_frame)
d = pd.read_csv(test_combined_frame)
# Create test submissions frame

def iditer(test_quiz_results):
    for i in test_quiz_results.id.tolist():
        yield i 
        
fakeids = iditer(test_quiz_results)

submissions_frame = pd.read_csv(test_submissions_frame)
# cut down to match other frame size
submissions_frame = submissions_frame[:len(test_quiz_results)]


from faker import Faker
import random
fake = Faker()

# quiz_id = random.randint( 11111, 99999 )
quiz_id = prev_quiz
submissions_frame.quiz_id = quiz_id
submissions_frame.course_id = test_course_id
submissions_frame.user_id = submissions_frame.apply(lambda x: next(fakeids), axis=1)
submissions_frame.submission_id = submissions_frame.apply(lambda x: fake.ean(length=8), axis=1)
submissions_frame.validation_token = submissions_frame.apply(lambda x: fake.sha1(), axis=1)
# submissions_frame.drop(['_requester', 'attributes', 'html_url', 'result_url'], inplace=True, axis=1)
submissions_frame.id = submissions_frame.apply(lambda x: fake.ean(length=8), axis=1)

submissions_frame


# attic

In [ ]:
def get_all_reports(course_id, quiz_id):
    """
     GET /api/v1/courses/:course_id/quizzes/:quiz_id/reports 
     """
    url = make_url(course_id, 'quizzes')
    url = "{}/{}/reports".format(url, quiz_id)
    print("getting ", url)
    response = requests.get(url, headers=make_request_header())
    return response
#     return send_get_request(url)



# r = get_all_reports(course_id, main_quiz)
r = get_all_reports(course_id, review_quiz)
# r = get_all_reports(course_id, meta_review)
# r = get_all_reports(prev_course_id, prev_quiz)
r.content

In [ ]:

import json

def request_report_creation(course_id, quiz_id):
    """
    ALWAYS RETURNS ERRORS
    First, we need to tell canvas to make the report 
    by hitting:  POST /api/v1/courses/:course_id/quizzes/:quiz_id/reports
    In theory this could return the report url
    """
    url = make_url(course_id, 'quizzes')
    url = "{}/{}/reports".format(url, quiz_id)

    data = {"quiz_report" : [{"report_type":"student_analysis"}],
            "include":["progress","file"]
           }
    headers = { 'Authorization': 'Bearer {}'.format( environment.CONFIG.canvas_token ),
              'Content-Type' : 'application/json'
              }
    data = json.dumps(data)
    result = requests.request("POST", url, data=data, headers=headers)
#     result = send_post_request(url, data)
    return result


def get_report_urls(course_id, quiz_id):
    """Now that the report, hopefully, exists, we can try to get the 
    urls to it
    """
    url = make_url(course_id, 'quizzes')
    url = "{}/{}/reports".format(url, quiz_id)
    
    print("Course {} Quiz {} : Requesting {}".format(course_id, quiz_id, url))

    data = {
#         "quiz_report":[{"report_type":"student_analysis","includes_all_versions":False}],
            "include":["progress","file"]
           }
    result = send_get_request(url, data)
    print(result)
    
    if len(result) == 1:
        result = [result]

    # Get the right type of report
    reports = [r for r in result if r['report_type'] == 'student_analysis']
    print("{} report urls received".format(len(reports)))
    urls = [r['url'] for r in reports]
    return urls

In [ ]:
# r = request_report_creation(course_id, main_quiz)
r = request_report_creation(course_id, review_quiz)
# r = request_report_creation(course_id, meta_review)
# r = request_report_creation(prev_course_id, prev_quiz)
r.status_code

In [ ]:
# Get the url to the report for the quiz

# report_urls = get_report_urls(course_id, main_quiz) #specified resource dn exist
report_urls = get_report_urls(course_id, review_quiz) #works
# report_urls = get_report_urls(course_id, meta_review) #works; also returns item_analysis report
# report_urls = get_report_urls(prev_course_id, prev_quiz) #works; also returns item_analysis report

report_url = report_urls[0]

print("Link to report : ", report_url)

In [ ]:
def utc_string_to_local_dt(utc_string):
    """'2020-02-07T07:59:59Z'
    returns Timestamp('2020-02-06 23:59:59-0800', tz='US/Pacific')
    """
    return pd.to_datetime(utc_string).tz_convert('US/Pacific')

In [ ]:
# import time
# def report_url_gen(starting_report_url):
#     """
#     Returns the original url followed by copies with the report id incremented by 1
#     starting_report_url: 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165098/reports/92640'
#     """
#     r = starting_report_url.split('/')
#     url_stub ="/".join(r[:-1])
#     rid = int(r[-1:][0])

#     while True:
#         yield "{}/{}?include[]=progress&include[]=file".format(url_stub, rid)
#         rid += 1
    


# def parse_download_url(response):
#     """Takess a requests.response from get_report_download_url
#     and tries to parse out the file download url.
#     todo: set this up to poll for progress 
#     """
#     c = json.loads(response.content)

#     try:
#         download_url = c['file']['url']
#         print(download_url)
#         return download_url
    
#     except KeyError:
#         # todo Poll the progress url until have download url
# #         progress_url = c['progress_url']
#         print("Error: No download url provided \n", c)


# def get_report_download_url(report_url):
#     """Uses the report url to get the url from which the csv file can be downloaded"""
# #     data = {"include": ["progress", "file"]}
# #     data = {"include": "file"}
#     url = "{}?include[]=progress&include[]=file".format(report_url)
#     data = {}

#     try:
#         # Make the request
#         # We can't use the usual function since we don't want response.json()
#         response = requests.get(report_url, headers=make_request_header())

#         print('response was ', response.status_code)
#         response.raise_for_status()
#         # todo or put check the progress url until workflow_state="completed"
# #         if response.progress.workflow_state != 'completed':
            
#         # Continuing on since was successful
#         return parse_download_url(response)
        
#     except HTTPError as http_err:
#         print(f'HTTP error occurred: {http_err}')  # Python 3.6

#     except Exception as err:
#         print(f'Other error occurred: {err}')  # Python 3.6



# def download_report(download_url, save_file_path=None):
#     """Once we have the url to download the csv file, this
#     will handle the download and load the data into a pandas
#     dataframe
#     """
#     # Request the results file for assignment
#     # Open it with pandas
#     frame = pd.read_csv(download_url)

#     if save_file_path:
#         # Temporary location to store the downloaded file
#         frame.to_csv(save_file_path)

#     return frame


# def retrieve_quiz_data(quiz, rest_timeout=60, max_id_attempts=20):
#     """Returns a dataframe of the student report
#     ACTUALLY WORKING VERSION!
#     WILL ONLY WORK IF THE GENERATE REPORT BUTTON HAS BEEN MANUALLY CLICKED FIRST
#     Returns:
#         DataFrame
        
#     """
#     # This should request that the reports be generated
#     reports = quiz.get_all_quiz_reports()
    
#     print("Resting for {} seconds while waiting for canvas to generate report".format(rest_timeout))
#     time.sleep(rest_timeout)
    
#     # The first report should be the student_analysis report. However, 
#     # for some reason, canvas will not return the file info with the 
#     # report we just created. However, it seems that the creation process 
#     # also created a report with a different id, which will contain the file info
#     # At least in testing, this report id was 2 more than the student report
#     # (the item_analysis report id was the student_analysis id + 1)
#     # But will use a generator to cover more possibilities
#     url_gen = report_url_gen(reports[0].url)
    
#     for _ in range(0, max_id_attempts):
#         # first value out of generator will be the original url
#         url = next(url_gen)
#         print("trying: ", url)
#         # We request the hidden report object which will have the url for downloading
#         # and parse out the url
#         download_url = get_report_download_url(url)
#         # We load the url and parse into a dataframe
#         if download_url:
#             return download_report(download_url)


# def save_downloaded_report(activity, frame):
#     """If we've downloaded the report programmatically, this 
#     saves it to the expected location
#     """
#     # save to file
#     create_folder(activity.folder_path)
#     try:
#         # if there's a particular section
#         fp = "{}/{}-student-work.csv".format(activity.folder_path, SECTION )
#     except NameError:
#         fp = "{}/{}-{}-student-work.csv".format(activity.folder_path, getDateForMakingFileName(), activity.safe_name )

#     try:
#         frame.to_csv(fp)
#     except Exception as e:
#         print("Error saving student work to file ", e)

from CanvasHacks.QuizReportFileTools import retrieve_quiz_data, save_downloaded_report

def make_quiz_repo( course, activity, save=True):
    """Gets all student work data for the activity that's part of the assignment
    loads it into a QuizRepository and 
    returns the repository
    """
    # Get quiz submission objects
    if isinstance(activity, Review):
        repo = ReviewRepository(activity, course)
    else:
        repo = QuizRepository(activity, course)
    
    # The InitialWork object will have the assignment id set as its id
    # if the type of assignment is a quiz, we will need the quiz id
    # That matters because it seems we can't use activity.id for topical assignment
    # because the object returned will be a quiz report 
#     assign = course.get_assignment(activity.id)
#     if assign.is_quiz_assignment:
#         quiz_id = assign.quiz_id
#         quiz = course.get_quiz(quiz_id)
#     quiz = course.get_quiz(activity.quiz_id)

    # Download submissions
    subRepo = QuizSubmissionRepository(repo.quiz)
#     submissions = quiz.get_submissions()    
    
    # the canvasapi has returned a bunch of objects
    # so we convert to dicts to make easier to create dataframe
#     submissions = [l.__dict__ for l in list(submissions)]

    # Download student work
    # This will work if the 'Create Report' button has been manually clicked
    student_work_frame = retrieve_quiz_data(repo.quiz)
    
    if save:
        save_downloaded_report(activity, student_work_frame)
    
#     if isinstance(activity, Review):
#         repo = ReviewRepository(activity, course)
#     else:
#         repo = QuizRepository(activity)
    
    # Doing the combination with submissions after saved to avoid 
    # mismatches of new and old data
    repo._process(student_work_frame, subRepo.frame)
    
    return repo

In [ ]:
send_get_request(url.format(main_quiz), { 'include': 'submission'})


In [ ]:
download_url = parse_download_url(response)

response_content_for_testing = b'{"id":91464,"report_type":"student_analysis","readable_type":"Student Analysis","includes_all_versions":false,"includes_sis_ids":true,"generatable":true,"anonymous":false,"url":"https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464","progress_url":"https://canvas.csun.edu/api/v1/progress/335965","created_at":"2019-12-27T01:45:29Z","updated_at":"2019-12-27T01:55:54Z","file":{"id":7868563,"uuid":"VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","folder_id":null,"display_name":"Quiz1-review Survey Student Analysis Report.csv","filename":"quiz_student_analysis_report.csv","upload_status":"success","content-type":"text/csv","url":"https://canvas.csun.edu/files/7868563/download?download_frd=1\\u0026verifier=VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","size":132,"created_at":"2019-12-27T01:55:54Z","updated_at":"2019-12-27T01:55:54Z","unlock_at":null,"locked":false,"hidden":false,"lock_at":null,"hidden_for_user":false,"thumbnail_url":null,"modified_at":"2019-12-27T01:55:54Z","mime_class":"file","media_entry_id":null,"locked_for_user":false},"quiz_id":165820}'

In [ ]:
data = {
#     'quiz_report': [{
#         'report_type': 'student_analysis',
#     }],
    "include": ["progress", "file"]
}

# This report url works properly!!!!!!!!!!!!!!!!
report_url = 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464'
# Make the request 
# We can't use the usual function since we don't want response.json()
head = { 'Authorization': 'Bearer {}'.format( environment.CONFIG.canvas_token ) }
response = requests.get( report_url, headers=make_request_header(), json=data )

print('response was ', response.status_code)


In [ ]:
# report_url = 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464'

response.content

__Example of what looking for__

send_get_request('https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464', {'include' : 'file'})

{'id': 91464,
 'report_type': 'student_analysis',
 'readable_type': 'Student Analysis',
 'includes_all_versions': False,
 'includes_sis_ids': True,
 'generatable': True,
 'anonymous': False,
 'url': 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464',
 'progress_url': 'https://canvas.csun.edu/api/v1/progress/335965',
 'created_at': '2019-12-27T01:45:29Z',
 'updated_at': '2019-12-27T01:55:54Z',
 'file': {'id': 7868563,
  'uuid': 'VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo',
  'folder_id': None,
  'display_name': 'Quiz1-review Survey Student Analysis Report.csv',
  'filename': 'quiz_student_analysis_report.csv',
  'upload_status': 'success',
  'content-type': 'text/csv',
  'url': 'https://canvas.csun.edu/files/7868563/download?download_frd=1&verifier=VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo',
  'size': 132,
  'created_at': '2019-12-27T01:55:54Z',
  'updated_at': '2019-12-27T01:55:54Z',
  'unlock_at': None,
  'locked': False,
  'hidden': False,
  'lock_at': None,
  'hidden_for_user': False,
  'thumbnail_url': None,
  'modified_at': '2019-12-27T01:55:54Z',
  'mime_class': 'file',
  'media_entry_id': None,
  'locked_for_user': False},
 'quiz_id': 165820}

while(1);{"id":91464,"report_type":"student_analysis","readable_type":"Student Analysis","includes_all_versions":false,"includes_sis_ids":true,"generatable":true,"anonymous":false,"url":"https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464","progress_url":"https://canvas.csun.edu/api/v1/progress/335965","created_at":"2019-12-27T01:45:29Z","updated_at":"2019-12-27T01:55:54Z","file":{"id":7868563,"uuid":"VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","folder_id":null,"display_name":"Quiz1-review Survey Student Analysis Report.csv","filename":"quiz_student_analysis_report.csv","upload_status":"success","content-type":"text/csv","url":"https://canvas.csun.edu/files/7868563/download?download_frd=1\u0026verifier=VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","size":132,"created_at":"2019-12-27T01:55:54Z","updated_at":"2019-12-27T01:55:54Z","unlock_at":null,"locked":false,"hidden":false,"lock_at":null,"hidden_for_user":false,"thumbnail_url":null,"modified_at":"2019-12-27T01:55:54Z","mime_class":"file","media_entry_id":null,"locked_for_user":false},"quiz_id":165820}

In [ ]:
params = { 'include' : ['submission_comments', 'user']}

In [ ]:
def download_submissions(course_id, quiz_id):
    """This isn't really needed"""
    course = canvas.get_course(course_id)
    quiz = course.get_quiz(quiz_id)
    submissions = quiz.get_submissions()
    submissions_frame = pd.DataFrame([l.__dict__ for l in list(submissions)])
    return submissions_frame

if TEST:
    if USE_PREVIOUS:
        # test -- using previous course
        submissions_frame = download_submissions(prev_course_id, prev_quiz)
    else:
        # test
        submissions_frame = pd.read_csv(test_submissions_frame)
# submissions_frame

In [ ]:
if TEST:

    if USE_PREVIOUS:
    #     or, if testing on past class
        all_quizzes = {q['id'] : q for q in get_all_course_quizzes(prev_course_id)}
        iq = all_quizzes.get(prev_quiz)
        initial = InitialWork(**iq)
        initial_work_repo = make_quiz_repo(initial)
        initial_work_repo 
        
    else:
        # Testing 
        initial_work_repo = QuizRepository(unit.initial_work)
        initial_work_repo.data = pd.read_csv(test_combined_frame)
        initial_work_repo._cleanup_data()
#         initial_work_repo._process(student_work_frame, submissions_frame)

else:
    # real
    initial_work_repo = make_quiz_repo(unit.initial_work)


In [ ]:
if not TEST:
    # Real version
    course = canvas.get_course(COURSE_ID)
    unit = Unit(course, UNIT_NUMBER)


#     # Download canvas data
#     all_quizzes = {q['id'] : q for q in get_all_course_quizzes(COURSE_ID)}

#     # Use canvas data to make objects defining each part of the assignment
#     iq = all_quizzes.get(part_ids[0])
#     initial = InitialWork(**iq)

#     rq = all_quizzes.get(part_ids[1])
#     review = Review(**rq)
#     review.access_code = codeRepo.review_code

#     mq = all_quizzes.get(part_ids[2])
#     meta = MetaReview(**mq)
#     meta.access_code = codeRepo.metareview_code

#     assignment = Assignment( initial, review, meta )

else:
    # Test version 

    from tests.factories.PeerReviewedFactories import *
    from tests.factories.ModelFactories import *
    test_data = test_data_factory()

    initial = InitialWork( **test_data[ 'initial' ] )
    initial.id = main_quiz
    # todo: add the quiz data into initial
    initial.question_columns = ['c1', 'c2']

    # todo should also be setting name, access code and link
    review = Review( **test_data[ 'review' ] )
    review.id = review_quiz
    review.access_code = codeRepo.review_code

    meta = MetaReview( **test_data[ 'metareview' ] )
    meta.id = meta_review
    meta.access_code = codeRepo.metareview_code
    
    unit = Unit({}, UNIT_NUMBER)
    unit.components = [ initial, review, meta]
#     unit = Assignment( initial, review, meta )